In [ ]:
import pandas as pd
import sqlalchemy as sql
import psycopg2
from sql_functions import get_engine
from sql_functions import get_sql_config
from sql_functions import push_to_database
import update as up

In [ ]:
df_budget = pd.read_excel('data/brb_2022.xlsm')

In [ ]:
df_budget.columns = df_budget.columns.str.replace(" ", "_")
df_budget.columns = df_budget.columns.str.lower()

In [ ]:
df_budget.rename(columns = {'bezeichnung':'ep_txt', 'bezeichnung.1':'kapitel_txt', 'bezeichnung_fz' : 'fz_txt' }, inplace = True)

In [ ]:
df_budget['ep'] = df_budget['ep'].astype(str)
df_budget['ep'] = df_budget['ep'].apply(lambda x:x.zfill(2))
df_budget['kapitel'] = df_budget['kapitel'].astype(str)
df_budget['kapitel'] = df_budget['kapitel'].str[-3:]
df_budget['titel'] = df_budget['titel'].astype(str)
df_budget['titel'] = df_budget['titel'].apply(lambda x:x.zfill(5))
df_budget['fz'] = df_budget['fz'].astype(str)
df_budget['fz'] = df_budget['fz'].apply(lambda x:x.zfill(3))


In [ ]:
df_budget.dropna(subset=['ansatz_2022_in_eur'],inplace=True)
df_budget.ansatz_2022_in_eur = df_budget.ansatz_2022_in_eur.apply(lambda x: int(x))


In [ ]:
df_budget["gruppe"] = df_budget["titel"].str[:3]
df_budget["counter"] = df_budget["titel"].str[3:]

In [ ]:
df_budget = df_budget.drop(['tgr./mgr.','bezeichnung.2','titel','ansatz_2021_in_eur','ea'], axis=1)

In [ ]:
df_budget['ansatz_2022_in_eur'] =df_budget[['ansatz_2022_in_eur', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)

In [ ]:
df_budget.insert(len(df_budget.columns),"jahr", 2022)
df_budget.insert(len(df_budget.columns),"state_id", 4)

In [ ]:
df_budget

In [ ]:
df_fz = df_budget[['fz','fz_txt']].copy()
df_fz.drop_duplicates(inplace = True)

In [ ]:
df_ep_txt = df_budget[['ep','ep_txt','jahr','state_id']].copy()
df_ep_txt.drop_duplicates(inplace=True)

In [ ]:
df_kapitel_txt = df_budget[['ep','kapitel','kapitel_txt','jahr','state_id']].copy()
df_kapitel_txt.drop_duplicates(inplace=True)

upload bundeslaender brb

In [ ]:
df_fz.rename(columns={"fz":"fkz", "fz_txt":"fkz_txt"}, inplace=True)

In [ ]:
get_sql_config()
engine = get_engine()


In [ ]:
up.update_fkz(df_fz)

In [ ]:
df_ep_txt.rename(columns={"jahr":"year"}, inplace=True)

In [ ]:
table_name = 'einzelplaene'
up.update_sql_table(df_ep_txt,table_name)
#push_to_database(df_ep_txt, "einzelplaene", engine, "capstone_public_budgeting")

In [ ]:
df_ep_txt

In [ ]:
df_kapitel_txt.rename(columns={"jahr":"year"}, inplace=True)

In [ ]:
table_name = 'kapitel'
up.update_sql_table(df_kapitel_txt,table_name)

#push_to_database(df_kapitel_txt, "kapitel", engine, "capstone_public_budgeting")

In [ ]:
#df_budget = df_budget.drop(['ep_txt', 'kapitel_txt', 'fz_txt'], axis=1)
df_budget.rename(columns={'ansatz_2022_in_eur':'amount', 'fz':'fkz', 'jahr':'year'}, inplace=True)
df_budget

In [ ]:
#push_to_database(df_budget, "budget_all", engine, "capstone_public_budgeting")